In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import csv
import dateutil
%pylab inline

Populating the interactive namespace from numpy and matplotlib


###Complaints Data (Manhattan, 2015)

In [2]:
complaints = pd.read_csv('Housing_Maintenance_Code_Complaints.csv', parse_dates=['ReceivedDate', 'StatusDate'])

In [3]:
complaints = complaints[complaints['BoroughID']==1]
complaints['ReceivedYR'] = complaints['ReceivedDate'].map(lambda x: x.year)
complaints_2015 =  complaints[complaints['ReceivedYR']==2015]

###The complaints data of Manhattan, 2015 are _"88,118"_

In [4]:
print len(complaints_2015)
print complaints_2015.dtypes

88118
ComplaintID                int64
BuildingID                 int64
BoroughID                  int64
Borough                   object
HouseNumber               object
StreetName                object
Zip                      float64
Block                      int64
Lot                        int64
Apartment                 object
CommunityBoard             int64
ReceivedDate      datetime64[ns]
StatusID                   int64
Status                    object
StatusDate        datetime64[ns]
ReceivedYR                 int64
dtype: object


In [5]:
#complaints_2015.sort(['ReceivedDate'])[:3]

### New columns ['BBL', 'Full_Address'] to match data in building level or BBL level
__LOGIC__
* BBL = BoroughID + Block + Lot
* Full_Address = HouseNumber + StreeName + 'New York, NY' + Zip
#Don't worry about the python warning (red box) which is just copy and paste columns issue. 

In [6]:
#Covert dtype int to str
complaints_2015['BoroughID'] = complaints_2015['BoroughID'].astype(str)
complaints_2015['Block'] = complaints_2015['Block'].astype(str)
complaints_2015['Lot'] = complaints_2015['Lot'].astype(str)
complaints_2015['Zip'] = complaints_2015['Zip'].astype(str)
complaints_2015['Zip'] = complaints_2015['Zip'].map(lambda x: str(x)[0:5])

#Add leading zeros to the Block and Lot (to make 10 digits BBL)
complaints_2015['Block'] = complaints_2015['Block'].apply(lambda x: x.zfill(5))
complaints_2015['Lot'] = complaints_2015['Lot'].apply(lambda x: x.zfill(4))

#Combime columns 
complaints_2015['BBL'] = complaints_2015['BoroughID'] + complaints_2015['Block'] + complaints_2015['Lot']
complaints_2015['Full_Address'] = complaints_2015['HouseNumber'] + ' ' + complaints_2015['StreetName'] + ' New York, NY ' + complaints_2015['Zip']

C:\Users\Boyeong\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
C:\Users\Boyeong\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Boyeong\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [7]:
print complaints_2015.Full_Address[:3]

174927                    442 2 AVENUE New York, NY 10010
174928    2155 ADAM C POWELL BOULEVARD New York, NY 10027
174929           1528 AMSTERDAM AVENUE New York, NY 10031
Name: Full_Address, dtype: object


###Geocoding for address matching in Bldg level (daily limitation --> will figure out)

In [8]:
#import geocoder

# Function to send address to geo-encode from Google API
#def geoencode(address):
    #g = geocoder.google(address) # ex. 30-27 Greenpoint Avenue LIC, NY
    #loc = [g.lat, g.lng, g.geometry] # create a list
    #return loc

#address = []
#lat = []
#lng = []
#geometry = []

#for i in complaints_2015.Full_Address:
    #g = geoencode(i) # call geoencode function
    #address.append(i)
    #lat.append(g[:1])
    #lng.append(g[1:2])
    #geometry.append(g[2:3])

#print "Finished Geocoding"

#Lat = pd.DataFrame(lat)
#Lat.columns=['Lat']
#Lon = pd.DataFrame(lng)
#Lon.columns=['Lon']
#Geometry = pd.DataFrame(geometry)
#Geometry.columns=['Geometry']

#complaints_2015 = pd.concat([complaints_2015, Lat, Lon, Geometry], axis=1)
#result.to_csv('e_waste_dataset_recyclingfacilities_geocoding_nyc.csv')

###Violations Data (Manhattan, 2015)

In [ ]:
violations = pd.read_csv('Housing_Maintenance_Code_Violations.csv', parse_dates=['InspectionDate', 'ApprovedDate']) # violations

In [ ]:
violations = violations[violations['BoroID']==1]
violations['InspectionYR'] = violations['InspectionDate'].map(lambda x: x.year)
violations_2015 =  violations[violations['InspectionYR']==2015]

###The violations data of Manhattan, 2015 are _"112,574"_
### !!! violations - complaints = 112,574 - 88,118 = 24,456

In [ ]:
print len(violations_2015)
print violations_2015.dtypes

### New columns ['BBL', 'Full_Address'] to match data in building level or BBL level
__LOGIC__
* BBL = BoroID + Block + Lot
* Full_Address = HouseNumber + StreeName + 'New York, NY' + Zip
#Don't worry about the python warning (red box) which is just copy and paste columns issue. 

In [ ]:
#Covert dtype int to str
violations_2015['BoroID'] = violations_2015['BoroID'].astype(str)
violations_2015['Block'] = violations_2015['Block'].astype(str)
violations_2015['Lot'] = violations_2015['Lot'].astype(str)
violations_2015['Zip'] = violations_2015['Zip'].astype(str)
violations_2015['Zip'] = violations_2015['Zip'].map(lambda x: str(x)[0:5])

#Add leading zeros to the Block and Lot (to make 10 digits BBL)
violations_2015['Block'] = violations_2015['Block'].apply(lambda x: x.zfill(5))
violations_2015['Lot'] = violations_2015['Lot'].apply(lambda x: x.zfill(4))

#Combime columns 
violations_2015['BBL'] = violations_2015['BoroID'] + violations_2015['Block'] + violations_2015['Lot']
violations_2015['Full_Address'] = violations_2015['HouseNumber'] + ' ' + violations_2015['StreetName'] + ' New York, NY ' + violations_2015['Zip']

In [ ]:
print violations_2015[:3]

In [ ]:
#save as csv (complaints)
#complaints_2015.to_csv('complaints_2015.csv')

In [ ]:
#save as csv (violations)
#violations_2015.to_csv('violations_2015.csv')

In [ ]:
print complaints_2015.sort(['ReceivedDate'])[:10]

In [ ]:
print complaints_2015.ReceivedDate[complaints_2015['Full_Address']=='166 WEST 87 STREET New York, NY 10024']
print '\n'
print violations_2015.InspectionDate[violations_2015['Full_Address']=='166 WEST 87 STREET New York, NY 10024']

###Data group by same address to compare C vs. V (simle numeric gap)

In [ ]:
#count of complaints and violations on same address (To compare the number of gap)
c_a = complaints_2015['Full_Address'].value_counts().to_frame()
v_a = violations_2015['Full_Address'].value_counts().to_frame()

In [ ]:
c_a.columns =['Complaints']
v_a.columns = ['Violations']
print c_a.head()
print v_a.head()
print '\n'
print len(c_a)
print len(v_a)

In [ ]:
#Join data C vs. V
c_v_a = pd.concat([c_a, v_a], axis=1)
len(c_v_a)

In [ ]:
#Calculate gap between complaints and violations 
#Positive no. means higher complaints but less violateions (may be good reporting?)
#Negative no. means less complaints but higher violations 
c_v_a['Gap']=c_v_a['Complaints'] - c_v_a['Violations']

In [ ]:
c_v_a.head(30)

#What I (we) want to do next...
* Plot timeseries
* Geocoding (point mapping + kernel mapping)
*